# Using subspaces

Often times, we encounter operators that have some conservation law. `dynamite` can take advantage of certain conservation laws by working in a restricted subspace.

**Note**: for performance reasons, dynamite does not currently check that your operator actually conserves the subspace you assign to it. Make sure you are using the right conservation law!

## SpinConserve

The first conservation law we will look at is conservation of total magnetization. One of our favorite models with this symmetry is the Heisenberg model:

$$H = \sum_{i,j} \vec{S}_i \cdot \vec{S}_j$$

where $\vec{S} = (S^x, S^y, S^z)$. 

Let's implement it:

In [ ]:
# always a good idea to set the spin chain length globally
from dynamite import config
config.L = 24

In [ ]:
from dynamite.operators import sigmax, sigmay, sigmaz, index_sum

def heisenberg():
    paulis = [sigmax, sigmay, sigmaz]
    return index_sum(sum(0.25*p(0)*p(1) for p in paulis))  # 0.25 to account for the factors of 1/2 in each spin operator!

# let's run the function and see what we get
H = heisenberg()
H

Looks pretty good! What is the dimension of the matrix corresponding to this operator?

In [ ]:
H.dim

A dimension $2^{24} \sim 16$ million square matrix. Even though it's sparse, storing that matrix would probably make your laptop pretty unhappy. 

(You may be wondering: why isn't my laptop *already* unhappy, didn't I just build the matrix? In fact no, dynamite delays building the matrix until it needs to, for example when you use it in a computation).

Let's see what happens if we instead switch to working in the total magnetization conserving subspace:

In [ ]:
from dynamite.subspaces import SpinConserve

In [ ]:
# L is total spins, k is the number of "up" spins
# here we work in the half filling symmetry sector 
H.subspace = SpinConserve(L=config.L, k=config.L//2)
H.dim

The dimension has been reduced by a factor of more than 6! In fact, it has been reduced to 24 choose 12, which is what we would expect for total spin conservation:

In [ ]:
from scipy.special import binom
binom(24, 12)

Note that subspaces can be applied to State objects in addition to operators:

In [ ]:
from dynamite.states import State

In [ ]:
psi = State(state='U'*12 + 'D'*12,  # first half spins up, second half down
            subspace=SpinConserve(L=config.L, k=config.L//2))
len(psi)

Note that the product state we specify must be in the subspace:

In [ ]:
# this causes an error
psi = State(state='U'*11 + 'D'*13,  # first 11 spins up
            subspace=SpinConserve(L=config.L, k=config.L//2))

### The `spinflip` flag for the SpinConserve subspace

Finally, we have one last trick up our sleeve for reducing the dimension of our problem: the Heisenberg model has an *additional* $\mathbb{Z}_2$ (spin flip) symmetry in addition to the $U(1)$ total magnetization conservation. The SpinConserve subspace has a special flag for including such an extra conservation law:

In [ ]:
SpinConserve(L=config.L, k=config.L//2, spinflip='+')  # '+' means that we are in the symmetric symmetry sector, '-' for antisymmetric

In [ ]:
# exercise: create a state in the SpinConserve subspace with spinflip set
# and confirm that the global spinflip operator has the expected eigenvalue 
# (+1 for the symmetric subspace, -1 for antisymmetric)

# it's easiest to set the subspace globally so it will be applied to both
# the spinflip operator and the state you create. do it like this:
# config.subspace = ... TODO ...

# here is the global spinflip operator, to get you started
from dynamite.operators import index_product
global_spinflip_operator = index_product(sigmax())

# your code for computing the expectation value (eigenvalue in this case) here!
# don't forget to initialize your State object to something (a product state, or random is fine)


## Parity

The next subspace we'll examine is parity conservation. This means that the total number of up spins is not globally conserved, but is instead conserved mod 2.

For this, we'll use the following long-range XX+Z model:

In [ ]:
config.subspace = None  # clear out any leftover configured subspace from above

from dynamite.operators import sigmax, sigmaz, index_sum, op_sum

def XXZ():
    interaction = op_sum(index_sum(sigmax(0)*sigmax(i)) for i in range(1, config.L))
    uniform_field = 0.5*index_sum(sigmaz())
    return interaction + uniform_field

# look at an example. we still have L=24 set from above
XXZ()

We can see by inspection of the Hamiltonian's definition that the X terms are always two-body, meaning that parity is conserved in the Z product state basis. We can easily apply this subspace in dynamite:

In [ ]:
H = XXZ()

print('full space dimension:     ', H.dim)

from dynamite.subspaces import Parity
H.subspace = Parity('even')

print('parity subspace dimension:', H.dim)

As expected, the dimension was cut in half. If you want the symmetry sector which has an odd number of up spins, just pass 'odd' to the subspace.

## Explicit

The last subspace that we will discuss here is the Explicit subspace. This allows you to define your own basis of product states! You could you this, for example, to define a Rydberg basis in which no up spin is adjacent to another up spin, for some aribtrary connectivity.

Under the hood, dynamite represents product states as integers, where each bit of the integer represents a spin (0=up, 1=down). The least significant bit of the integer is spin 0. This is how we will pass our array of product states to the Explicit subspace.

As a simple example here, we will reproduce the total magnetization conservation subspace by hand!

In [ ]:
# let's use numpy's function that counts the number of set bits in an integer,
# to pick out all integers between 0 and 2^L-1 that have L//2 bits set to 1 
import numpy as np

# note: this toy example is very slow, you probably want to do something more clever 
# with e.g. numpy vectorization when creating your own list of states for Explicit subspaces!
config.L = 18
subspace_product_states = [x for x in range(2**config.L) if np.int64(x).bit_count() == config.L//2]

# show that these states all have half spins up:
for x in subspace_product_states[:10]:
    print(bin(x)[2:].zfill(config.L)) # print binary representation, with leading zeros
print('...')

Now that we have our list of product states for the basis, we simply call the Explicit subspace:

In [ ]:
from dynamite.subspaces import Explicit

In [ ]:
explicit_subsp = Explicit(subspace_product_states)
explicit_subsp.L = config.L  # config.L does not currently propagate to subspaces

# it's equivalent to the SpinConserve one! just made by hand
explicit_subsp == SpinConserve(config.L, config.L//2)

## Up next

[Continue to notebook 6](6-ShellMatrices.ipynb) to learn how to save memory through "matrix-free" computation. 